In [1]:
# # 03_ml_modeling.ipynb

# import sys
# sys.path.append("../src")
# from modeling import split_X_y, split_train_test, scale_features, train_simple_model, evaluate_model
# import pandas as pd

# # --- Load Processed CSV ---
# df = pd.read_csv("../data/processed/bank_transactions_features.csv")

# # --- Split features / target ---
# X, y = split_X_y(df, "Is_Fraud")

# # --- Train/Test Split ---
# X_train, X_test, y_train, y_test = split_train_test(X, y, stratify=y)

# # --- Scale numeric features ---
# X_train_s, X_test_s = scale_features(X_train, X_test)

# # --- Train Logistic Regression ---
# clf = train_simple_model(X_train_s.fillna(0), y_train, problem="classification", model_name="logistic")

# # --- Evaluate ---
# metrics = evaluate_model(clf, X_test_s.fillna(0), y_test)
# print(metrics)

# # --- Save Model ---
# import joblib
# joblib.dump(clf, "../model/logistic_model.pkl")


In [10]:
# import sys
# sys.path.append("../src/")

# import pandas as pd
# import joblib
# from modeling import split_X_y, split_train_test, scale_features, train_simple_model, evaluate_model

# # --- Load processed CSV (already one-hot encoded) ---
# FEATURES_PATH = "../data/processed/bank_transactions_features.csv"
# df = pd.read_csv(FEATURES_PATH)

# # --- Split X / y ---
# X = df.drop(columns=["Is_Fraud"])
# y = df["Is_Fraud"]

# # --- Save column order BEFORE training (for dashboard consistency) ---
# columns_used = X.columns.tolist()
# joblib.dump(columns_used, "../model/columns_used.pkl")

# # --- Train/Test split ---
# X_train, X_test, y_train, y_test = split_train_test(X, y, stratify=y)

# # --- Optional: Scale numeric features ---
# X_train_s, X_test_s = scale_features(X_train.fillna(0), X_test.fillna(0))

# # --- Train model (Random Forest with class balancing) ---
# clf = train_simple_model(
#     X_train_s,
#     y_train,
#     problem="classification",
#     model_name="random_forest",
#     random_state=42,
#     n_estimators=200,
#     max_depth=10,
#     class_weight="balanced"
# )

# # --- Evaluate ---
# metrics = evaluate_model(clf, X_test_s, y_test)
# print("Model Evaluation Metrics:", metrics)

# # --- Save trained model ---
# MODEL_PATH = "../model/rf_model.pkl"
# joblib.dump(clf, MODEL_PATH)
# print(f"Random Forest model saved at: {MODEL_PATH}")


In [11]:
# import sys
# sys.path.append("../src/")

# import pandas as pd
# import joblib
# from modeling import split_X_y, split_train_test, scale_features, train_simple_model, evaluate_model
# from imblearn.ensemble import BalancedRandomForestClassifier
# from sklearn.metrics import classification_report

# # --- Load processed CSV (already one-hot encoded) ---
# FEATURES_PATH = "../data/processed/bank_transactions_features.csv"
# df = pd.read_csv(FEATURES_PATH)

# # --- Split X / y ---
# X = df.drop(columns=["Is_Fraud"])
# y = df["Is_Fraud"]

# # --- Save column order BEFORE training (for dashboard consistency) ---
# columns_used = X.columns.tolist()
# joblib.dump(columns_used, "../model/columns_used.pkl")

# # --- Train/Test split ---
# X_train, X_test, y_train, y_test = split_train_test(X, y, stratify=y)

# # --- Optional: Scale numeric features ---
# X_train_s, X_test_s = scale_features(X_train.fillna(0), X_test.fillna(0))

# # --- Train model for imbalanced data ---
# clf = BalancedRandomForestClassifier(
#     n_estimators=300,
#     max_depth=10,
#     random_state=42,
#     replacement=True  # allows sampling with replacement
# )
# clf.fit(X_train_s, y_train)

# # --- Evaluate ---
# y_pred = clf.predict(X_test_s)
# print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))

# # ROC AUC for imbalanced data
# if hasattr(clf, "predict_proba"):
#     y_prob = clf.predict_proba(X_test_s)[:, 1]
#     from sklearn.metrics import roc_auc_score
#     auc = roc_auc_score(y_test, y_prob)
#     print(f"ROC AUC: {auc:.4f}")

# # --- Save trained model ---
# MODEL_PATH = "../model/rf_model_imbalanced.pkl"
# joblib.dump(clf, MODEL_PATH)
# print(f"Balanced Random Forest model saved at: {MODEL_PATH}")


In [12]:
# import sys
# import os
# import pandas as pd
# import joblib
# import numpy as np

# # Add src to path
# sys.path.append("../src/")

# from modeling import split_X_y, split_train_test, scale_features, train_simple_model, evaluate_model
# from imblearn.ensemble import BalancedRandomForestClassifier
# from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# # =====================================================
# # 1. SETUP & LOAD
# # =====================================================
# # Ensure model directory exists
# os.makedirs("../model", exist_ok=True)

# FEATURES_PATH = "../data/processed/bank_transactions_features.csv"
# print(f"Loading data from {FEATURES_PATH}...")
# df = pd.read_csv(FEATURES_PATH)

# # =====================================================
# # 2. SPLITTING
# # =====================================================
# # Use helper to split X and y
# X, y = split_X_y(df, target="Is_Fraud")

# # --- SAVE COLUMN METADATA ---
# # Crucial: Save the column names so the dashboard knows the exact expected input order
# cols_path = "../model/columns_used.pkl"
# joblib.dump(X.columns.tolist(), cols_path)
# print(f"Saved feature column list to {cols_path}")

# # Split Train/Test
# X_train, X_test, y_train, y_test = split_train_test(X, y, stratify=y)

# # =====================================================
# # 3. PREPROCESSING (Minimal for Trees)
# # =====================================================
# # Note: Tree-based models (Random Forest) do not require scaling.
# # We skip StandardScaling to simplify the inference pipeline.
# # We only ensure no NaNs exist (though clean_missing should have handled this).

# X_train = X_train.fillna(0)
# X_test = X_test.fillna(0)

# # =====================================================
# # 4. TRAINING
# # =====================================================
# print("Training BalancedRandomForestClassifier...")
# # BalancedRandomForest undersamples the majority class inside each bootstrap sample
# clf = BalancedRandomForestClassifier(
#     n_estimators=300,
#     max_depth=10,       # Kept distinct to prevent overfitting
#     min_samples_leaf=5, # Added for regularization
#     random_state=42,
#     replacement=True,
#     sampling_strategy='auto',
#     n_jobs=-1,          # Use all CPU cores
#     verbose=1
# )

# clf.fit(X_train, y_train)

# # =====================================================
# # 5. EVALUATION
# # =====================================================
# print("\n--- Evaluating Model ---")
# y_pred = clf.predict(X_test)
# y_prob = clf.predict_proba(X_test)[:, 1]

# # 1. Classification Report
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred, digits=4))

# # 2. ROC AUC
# auc = roc_auc_score(y_test, y_prob)
# print(f"ROC AUC Score: {auc:.4f}")

# # 3. Confusion Matrix (Raw counts)
# cm = confusion_matrix(y_test, y_pred)
# print("\nConfusion Matrix:")
# print(cm)
# print(f"False Positives: {cm[0][1]}")
# print(f"False Negatives: {cm[1][0]}")

# # =====================================================
# # 6. SAVE MODEL
# # =====================================================
# MODEL_PATH = "../model/rf_model_imbalanced.pkl"
# joblib.dump(clf, MODEL_PATH)
# print(f"\nModel saved successfully at: {MODEL_PATH}")

In [13]:
import sys
import os
import pandas as pd
import joblib

# Add src to path
sys.path.append("../src/")

# Import updated helpers
from modeling import split_X_y, split_train_test, scale_features, train_simple_model, evaluate_model

# =====================================================
# 1. SETUP & LOAD
# =====================================================
os.makedirs("../model", exist_ok=True)
FEATURES_PATH = "../data/processed/bank_transactions_features.csv"

print(f"Loading data from {FEATURES_PATH}...")
df = pd.read_csv(FEATURES_PATH)

# =====================================================
# 2. SPLITTING
# =====================================================
X, y = split_X_y(df, target="Is_Fraud")

# --- SAVE METADATA 1: Column Names ---
# The Streamlit app needs this to know the order of features
cols_path = "../model/columns_used.pkl"
joblib.dump(X.columns.tolist(), cols_path)
print(f"Saved feature column list to {cols_path}")

# Split Train/Test
X_train, X_test, y_train, y_test = split_train_test(X, y, stratify=y)

# =====================================================
# 3. PREPROCESSING (Scaling)
# =====================================================
# 1. Handle Missing Values first
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# 2. Scale Features & CAPTURE THE SCALER
# Even if Trees don't strictly need it, using a scaler allows you 
# to easily swap to Logistic Regression later without changing the pipeline.
print("Scaling features...")
scaler, X_train_scaled, X_test_scaled = scale_features(X_train, X_test, method="standard")

# --- SAVE METADATA 2: The Scaler ---
# CRITICAL: We save the math (mean/std) so the Streamlit app 
# can transform the user's single input exactly like the training data.
if scaler:
    scaler_path = "../model/scaler.pkl"
    joblib.dump(scaler, scaler_path)
    print(f"Saved scaler to {scaler_path}")

# =====================================================
# 4. TRAINING
# =====================================================
print("Training BalancedRandomForestClassifier via Helper...")

clf = train_simple_model(
    X_train_scaled,  # Use the SCALED data
    y_train,
    problem="classification",
    model_name="balanced_random_forest",
    resample=False, # Helper handles this internal logic
    # Specific Params
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=5,
    n_jobs=-1
)

# =====================================================
# 5. EVALUATION
# =====================================================
print("\n--- Evaluating Model ---")
metrics = evaluate_model(clf, X_test_scaled, y_test, problem="classification")
print(f"Metrics: {metrics}")

# =====================================================
# 6. SAVE MODEL
# =====================================================
MODEL_PATH = "../model/rf_model_imbalanced.pkl"
joblib.dump(clf, MODEL_PATH)
print(f"\nModel saved successfully at: {MODEL_PATH}")

Loading data from ../data/processed/bank_transactions_features.csv...


2025-12-15 14:32:37,696 - INFO - Split data: X.shape=(200000, 83), y.shape=(200000,)
2025-12-15 14:32:37,873 - INFO - Train-test split: X_train=(160000, 83), X_test=(40000, 83)


Saved feature column list to ../model/columns_used.pkl
Scaling features...


2025-12-15 14:32:37,957 - INFO - Scaled 5 columns using standard scaler.


Saved scaler to ../model/scaler.pkl
Training BalancedRandomForestClassifier via Helper...


2025-12-15 14:32:43,149 - INFO - Trained balanced_random_forest model for classification
2025-12-15 14:32:43,291 - INFO - Top 5 feature importances:
Transaction_Amount    0.129550
Account_Balance       0.102860
Age                   0.075354
Transaction_Day       0.065404
Transaction_Hour      0.063335
dtype: float64



--- Evaluating Model ---


2025-12-15 14:32:43,631 - INFO - Evaluation results: {'accuracy': 0.5232, 'precision': 0.048644929070506035, 'recall': 0.4554013875123885, 'f1': 0.08790052606408418, 'roc_auc': np.float64(0.4951332118667238)}


Metrics: {'accuracy': 0.5232, 'precision': 0.048644929070506035, 'recall': 0.4554013875123885, 'f1': 0.08790052606408418, 'roc_auc': np.float64(0.4951332118667238)}

Model saved successfully at: ../model/rf_model_imbalanced.pkl
